<center><a href="http://www.cosmostat.org/" target_="blank"><img src="./images/cosmostat_logo.png" width="400"></a></center>

# Introduction to Sparsity III: Deconvolution

---

> Author: <font color='#f78c40'>Samuel Farrens</font>    
> Year: 2018 (updated in 2023)    
> Email: [samuel.farrens@cea.fr](mailto:samuel.farrens@cea.fr)  
> Website: <a href="https://sfarrens.github.io" target="_blank">https://sfarrens.github.io</a>

## Contents

1. [Set-Up](#Set-Up)
1. [Introduction](#Introduction)
 * [Objective](#Objective)
 * [Point Spread Function](#Point-Spread-Function)
1. [Deconvolution Example](#Deconvolution-Example)
 * [Convolution](#Convolution)
 * [Deconvolution](#Deconvolution)
 * [Convolution with Noise](#Convolution-with-Noise)
 * [Deconvolution with Noise](#Deconvolution-with-Noise)
1. [Ill-Posed Problem](#Ill-Posed-Problem)
 * [Ill-Conditioned Problem](#Ill-Conditioned-Problem)
1. [Deconvolution Exercise](#Deconvolution-Exercise)
 * [Deconvolution with Sparsity](#Deconvolution-with-Sparsity)

## Set-Up

Here we will import a couple of packages that will be needed throughout the notebook. 

Users new to Jupyter notebooks should note that cells are executed by pressing <kbd>SHIFT</kbd>+<kbd>ENTER</kbd> (&#x21E7;+ &#x23ce;). See <a href="https://jupyter-notebook.readthedocs.io/en/stable/" target_="blanck">here</a> for more details.

In [ ]:
# Uncomment the following lines if you are running this on Google Colab
# !git clone -b ada https://github.com/CosmoStat/Tutorials.git
# import sys
# sys.path.insert(0,'/content/Tutorials')

In [ ]:
# Import the numpy package with the alias np.
import numpy as np

# Import tools from Scipy
from scipy.ndimage import gaussian_filter
from scipy.datasets import face

# Import tutorial functions.
from sparsity_tutorial import plot
from sparsity_tutorial.functions import *

## Introduction

### <font color='#007acc'>Objective</font>

The objective of this notebook is to demonstrate how sparsity can be used for deconvolution. A very simple image is used to simpify the implementation. More realistic applications to astrophysical images are provided in

* <a href="https://www.aanda.org/articles/aa/abs/2017/05/aa29709-16/aa29709-16.html" target_="blank">Farrens et al. (2017)</a>

### <font color='#007acc'>Point Spread Function</font>

All optical systems are subject to imperfections that distort the images obtained. The sum of these aberrations is commonly referred to as the point spread function (PSF) or instrumental response. Removing the effects of the PSF from images of interest if a fundamental aspect of image processing, particularly in fields such as astrophysics where accurate measurments of the shapes and flux of objects has a significant impact on the science.

The blurring introduced by the PSF can be modelled as a convolution of the PSF with the image. Consequently, the reverse process, deconvolution, removes this effect.

In the following cell we define a function for generating a Gaussian PSF.

In [ ]:
# Function to generate a Gaussian PSF.
def make_psf(shape, sigma):
    
    psf = np.zeros(shape)
    psf[tuple(np.array(psf.shape) // 2)] = 1
    
    return gaussian_filter(psf, sigma)

plot.display(make_psf((100, 100), 10), title='PSF')

## Deconvolution Example

### <font color='#007acc'>Convolution</font>

The <a href="https://en.wikipedia.org/wiki/Convolution_theorem" target_="blank">convolution theorem</a> shows that a signal $a$ can be convolved with a signal $b$ (under certain conditions) as follows

$$c = a * b = \mathcal{F}^{-1}(\mathcal{F}(a) \cdot \mathcal{F}(b))$$

We can implement this as follows.

In [ ]:
# This function convolves an image with a kernel using FFT.
def fftconvolve(image, kernel):

    x = np.fft.fftshift(np.fft.fftn(image))
    y = np.fft.fftshift(np.fft.fftn(kernel))

    return np.real(np.fft.fftshift(np.fft.ifftn(np.fft.ifftshift(x * y))))

The following cell demonstrates what happens to an image when it is convolved with the PSF we defined.

In [ ]:
# Load example image
image = face()[:700, :700, 0]

# Generate a PSF
psf_1 = make_psf(image.shape, 10)

# Convolve image with PSF
image_conv = fftconvolve(image, psf_1)

In [ ]:
# Display the original image
plot.display(image, cmap='Greys_r', title='Original Image')

In [ ]:
# Display the convolved image
plot.display(image_conv, cmap='Greys_r', title='Convolved Image')

### <font color='#007acc'>Deconvolution</font>

<a href="https://en.wikipedia.org/wiki/Deconvolution" target_="blank">Deconvolution</a> is the process of reversing the effects of convolution.

$$a = \mathcal{F}^{-1}\bigg(\frac{\mathcal{F}(c)}{\mathcal{F}(b)}\bigg)$$

We can implement this process as follows.

In [ ]:
# This function deconvolves an image with a kernel using FFT.
def fftdeconvolve(image, kernel):

    x = np.fft.fftshift(np.fft.fftn(image))
    y = np.fft.fftshift(np.fft.fftn(kernel))

    return np.real(np.fft.fftshift(np.fft.ifftn(np.fft.ifftshift(x / y))))

Using this function we can recover our original image given that we know the PSF.

In [ ]:
image_deconv = fftdeconvolve(image_conv, psf_1)

# Display the convolved image
plot.display(image_deconv, cmap='Greys_r', title='Deconvolved Image')

As you can see the all the blurring effects have been removed from the convolved image.

### <font color='#007acc'>Convolution with Noise</font>

In practical applications all observations contain some degree of noise. We can imagine that our Raccoon was photographed with a very good instrument with only the tiniest amount of noise ($\sigma=10^{-6}$) as follows.

In [ ]:
# Add Gaussian noise to convolved image
image_conv_noise = image_conv + np.random.randn(*image_conv.shape) * 1e-6

# Display the convolved image
plot.display(image_conv_noise, cmap='Greys_r', title='Noisy Convolved Image')

Given the amount of noise, the convolved image looks virtually the same.

### <font color='#007acc'>Deconvolution with Noise</font>

Let's see what happens when we try to recoever the original image from the noisy convolved image.

In [ ]:
image_deconv_noise = fftdeconvolve(image_conv_noise, psf_1)

# Display the convolved image
plot.display(image_deconv_noise, cmap='Greys_r', title='Deconvolved Image')

The image is completely destroyed! Why?

## Ill-Posed Problem

A problem is considered <a href="https://en.wikipedia.org/wiki/Well-posed_problem" target_="blank">well-posed</a> if 

1. a solution exists,
2. the solution is unique,
3. the solution's behavior changes continuously with the initial conditions.

Any problem that does meet all three of this requirements is considered "ill-posed".

### <font color='#007acc'>Ill-Conditioned Problem</font>

A problem (or matrix) with a large <a href="https://en.wikipedia.org/wiki/Condition_number" target_="blank">condition number</a> is considered to be "ill-coniditioned". This relates to the 3rd point on our list of requirements for a well-posed problem.

Take for example the following two problems

$$\begin{bmatrix} 4 \\ 7 \end{bmatrix} = \begin{bmatrix} 1 & 2 \\ 2 & 3 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$$

$$\begin{bmatrix} 3 \\ 1.47 \end{bmatrix} = \begin{bmatrix} 1 & 2 \\ 0.48 & 0.99 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$$

Both are linear inverse problems of the form $y=Hx$, which can be implemented as follows.

In [ ]:
# Data for first problem
y1 = np.array([4.0, 7.0])
H1 = np.array([[1.0, 2.0], [2.0, 3.0]])

# Data for second problem|
y2 = np.array([3.0, 1.47])
H2 = np.array([[1.0, 2.0], [0.48, 0.99]])

Two solve both problems we simply need to calculate $x = H^{-1}y$ as follows.

In [ ]:
# Solution to the first problem
x1 = np.linalg.inv(H1) @ y1
print('x1 =', x1)

# Solution to the second problem
x2 = np.linalg.inv(H2) @ y2
print('x2 =', x2)

Now, let's see what happens if we add a small change to $y$ (e.g. mimicking the effects of noise).

$$\begin{bmatrix} 4 \\ 7.01 \end{bmatrix} = \begin{bmatrix} 1 & 2 \\ 2 & 3 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$$

$$\begin{bmatrix} 3 \\ 1.48 \end{bmatrix} = \begin{bmatrix} 1 & 2 \\ 0.48 & 0.99 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}$$

In [ ]:
# Noisy versions of y
y1_noise = np.array([4.0, 7.01])
y2_noise = np.array([3.0, 1.48])

x1_noise = np.linalg.inv(H1) @ y1_noise
print('x1 =', x1_noise)

x2_noise = np.linalg.inv(H2) @ y2_noise
print('x2 =', x2_noise)

Notice that for the first problem the solution has changed only as much as the input data, while for the second problem the solution is completely different. 

It is clear from these results that the first problem is well-conditioned well the second is ill-conditioned. This means that the first problem can be solved analytically, even with noise, while the second cannot.

We can relate this property back to our deconvolution problem, which can also be modelled as a linear inverse problem of the form

$$y=Hx + n$$

where $y$ is the noisy convolved image we observe, $H$ is a operator that represents convolution with the PSF, $x$ is the original image and $n$ is Gaussian random noise. As $H$ is ill-conditioned and hence the problem is ill-posed this cannot be solved analytically. This is where sparsity comes in!


## Deconvolution Exercise

### <font color='#007acc'>Deconvolution with Sparsity</font>

Given that we can model our observations as $y=Hx + n$, we aim to solve an optimisation problem of the form

$$\begin{aligned} & \underset{\alpha}{\text{argmin}} & \frac{1}{2}\|y-H\phi\alpha\|_2^2 + \lambda\|\alpha\|_1\end{aligned}$$

where we assume we know that our data is sparse in the dictionary $\phi$. Notice that this is exactly the same problem we needed to solve for the CS exercises, only now $H$ is a convolution operator rather than a masking operator.

### <font color='#007acc'>Problem</font>

An observer with has obtained a very blurry and noisy image with instrument. Fortunately she is celever and knows the exact PSF of her instrument. She now provides you with her image and the PSF her instrument and asks for help in deconvoving the PSF effects.

You, being an expert in sparsity and having some experience with this type of data, know that this signal is sparse in Fourier space. You also know that $\lambda=0.003$ would be an appropriate value for this problem.

**Questions and tasks**
1. Try to deconvolve the image analytically.
1. Does it work?
1. What is the gradient of this problem?
1. What is the cost function of this problem?
1. Make a first guess for $x$.
1. Deconvolve the image using the Forward-Backward algorithm.
1. Display your deconvolved image $\hat{x}$.
1. Roughly how many iterations did it take to converge?
1. Finally, load the file `data/cs_true_data.npy` to determine the NMSE of your solution.

### <font color='#007acc'>Data</font>

For this exercise you are given a noisy convolved observation $y$ and a PSF. Your task is to deconvolve the psf effects from the observation to recover the original image $x$. 

In [ ]:
# Load the observation.
y = np.load('data/deconv_obs_data.npy')

# Load the PSF.
psf = np.load('data/deconv_psf.npy')

In [ ]:
# Display the observed image.
plot.display(y, r'$y$')

In [ ]:
# Display the PSF.
plot.display(psf, 'PSF')

### <font color='#007acc'>Functions</font>

Some functions that may be useful for this exercise are provided below.

In [ ]:
# H operator.
def H(data, psf):
    
    return fftconvolve(data, psf)

# H transpose operator.
def Ht(data, psf):
    
    return fftconvolve(data, np.rot90(psf, 2))

In [ ]:
# Function that performs simple forward backward splitting.
def forwardBackward(observation, first_guess, psf, grad, lambda_val, n_iter=300, gamma=1.0, return_cost=False):
    
    alpha_rec = first_guess
    cost = []
            
    for _ in range(n_iter):
        
        alpha_temp = alpha_rec - gamma * grad(observation, alpha_rec, psf)
        alpha_rec = soft_thresh(alpha_temp, lambda_val)
        cost.append(cost_func(observation, alpha_rec, psf, lambda_val))
    
    if return_cost:
        return alpha_rec, cost
    else:
        return alpha_rec

### <font color='#007acc'>Solutions</font>

### <font color='#e42d2f'>EXERCISE</font>

Provide your solutions in the cell(s) below.

---

## <font color='#fdbf00'>Hints</font>

**Deconvolution Exercise**

1. I wonder if the functions provided will help for defining the cost function and the gradient.
1. Make sure you display your deconvolved image in the correct domain.
1. If you plot your cost function it should be pretty easy to see when the algorithm converged.

[Open next notebook ->](./wavelets_1.ipynb)